In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
from geopy.geocoders import Nominatim

import pandas as pd
import seaborn.apionly as sns
from datetime import date, datetime
from haversine import haversine

# statistics package
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# packages for mapping
from mpl_toolkits.basemap import Basemap

# packages for interactive graphs
from ipywidgets import widgets, interact
from IPython.display import display
from copy import deepcopy as copy
import time
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
geolocator = Nominatim()
findzip = ZipcodeSearchEngine()

In [3]:
historical_data = pd.read_csv('train.csv')

In [5]:
historical_data.head()

,Unnamed: 0,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,...,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,payment_type,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,...,40.757977,-73.989838,40.751171,6.5,CSH,0.0,0.5,0.0,0.0,7.0
1,1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,...,40.731781,-73.994499,40.750660,6.0,CSH,0.5,0.5,0.0,0.0,7.0
2,2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,...,40.737770,-74.009834,40.726002,5.5,CSH,1.0,0.5,0.0,0.0,7.0
3,3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,...,40.759945,-73.984734,40.759388,5.0,CSH,0.5,0.5,0.0,0.0,6.0
4,4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,...,40.748528,-74.002586,40.747868,9.5,CSH,0.5,0.5,0.0,0.0,10.5


## Data Preprocessing

In [6]:
process_train_data = copy(historical_data)

## Train Data

In [7]:
train_data = copy(process_train_data.loc[0:200000,])
del train_data['Unnamed: 0']
train_data.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,payment_type,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,...,40.757977,-73.989838,40.751171,6.5,CSH,0.0,0.5,0.0,0.0,7.0
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,...,40.731781,-73.994499,40.750660,6.0,CSH,0.5,0.5,0.0,0.0,7.0
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,...,40.737770,-74.009834,40.726002,5.5,CSH,1.0,0.5,0.0,0.0,7.0
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,...,40.759945,-73.984734,40.759388,5.0,CSH,0.5,0.5,0.0,0.0,6.0
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,...,40.748528,-74.002586,40.747868,9.5,CSH,0.5,0.5,0.0,0.0,10.5


In [8]:
train_data["pickup_datetime"] = pd.to_datetime(train_data["pickup_datetime"])
train_data["dropoff_datetime"] = pd.to_datetime(train_data["dropoff_datetime"])

In [9]:
train_data["pickup_day"] = train_data["pickup_datetime"].apply(lambda x: x.day)
train_data["pickup_weekday"] = train_data["pickup_datetime"].apply(lambda x: x.weekday())
train_data["pickup_hour"] = train_data["pickup_datetime"].apply(lambda x: x.hour)
train_data["pickup_minute"] = train_data["pickup_datetime"].apply(lambda x: x.minute)
train_data["pickup_time"] = train_data["pickup_hour"] + (train_data["pickup_minute"] / 60)
train_data["dropoff_hour"] = train_data["dropoff_datetime"].apply(lambda x: x.hour)

In [10]:
train_data['pickup_date'] = [date.date() for date in train_data['pickup_datetime']]
train_data['dropoff_date'] = [date.date() for date in train_data['dropoff_datetime']]

### The distance is calculated in kilometers

In [11]:
def distance(lat1, lon1, lat2, lon2):
    """calculates the Manhattan distance between 2 points
        using their coordinates
    
    Parameters
    ----------
    lat1: float
        latitude of first point
        
    lon1: float
        longitude of first point
        
    lat2: float
        latitude of second point
    
    lon2: float
        longitude of second point
        
    Returns
    -------
    d: float
        The Manhattan distance between the two points in kilometers
        
    """
    
    d = haversine((lat1, lon1), (lat2, lon1)) + haversine((lat2, lon1), (lat2, lon2))
    return d

In [12]:
train_data["distance"] = train_data.apply(lambda row: distance(row["pickup_latitude"], 
                                               row["pickup_longitude"], 
                                               row["dropoff_latitude"], 
                                               row["dropoff_longitude"]), axis=1)

### The speed is calculated in km/h

In [13]:
train_data["speed"] = train_data["distance"] / (train_data["trip_time_in_secs"] / 3600)

In [14]:
pickup_datetime = train_data['pickup_datetime']
day_interval = []
for i in range(pickup_datetime.size):
    hour = pickup_datetime[i].hour
    if hour>=6 and hour<12:
        day_interval.append(0)
    elif hour>=12 and hour<18:
        day_interval.append(1)
    elif hour>=18 and hour<24:
        day_interval.append(2)
    else:
        day_interval.append(3)
train_data['day_interval'] = day_interval

In [15]:
def add_waiting_time_and_penalty(train_data):
    sorted_train_data = copy(train_data.sort_values(by=['hack_license', 'pickup_datetime'], ascending=True))
    sorted_train_data['waiting_time'] = 0.0
    sorted_train_data['waiting_penalty'] = 0.0
    previous_license = None
    previous_drpoff_datatime = 0
    previous_date = None
    previous_row = None
    wait_time_list = []
    wait_penalty_list = []
    average_speed = sorted_train_data['speed'].mean()
    for index,row in sorted_train_data.T.iteritems():
        if row['hack_license'] == previous_license and row['pickup_date'] == previous_date:
            diff =  row['pickup_datetime'] - previous_row['dropoff_datetime']
            _distance = distance(row["pickup_latitude"], row["pickup_longitude"], previous_row["dropoff_latitude"], previous_row["dropoff_longitude"])
            speed = row['speed']
            if(speed <= 0):
                speed = average_speed
            time_take_to_reach = (_distance/speed)*3600
            if (time_take_to_reach>diff.total_seconds()):
                wait_time_list.append(0.0)
                wait_penalty_list.append(0.0)
            else:
                wait_time_list.append(diff.total_seconds()-time_take_to_reach)
                wait_penalty_list.append((diff.total_seconds()-time_take_to_reach)/3600*5)
        else:
            wait_time_list.append(0.0)
            previous_license = row['hack_license'] 
            previous_date = row['pickup_date']
            previous_row = row
            wait_penalty_list.append(0.0)
    sorted_train_data['waiting_time'] = wait_time_list
    sorted_train_data['waiting_penalty'] = wait_penalty_list
    return sorted_train_data

In [16]:
sorted_train_data = add_waiting_time_and_penalty(train_data)
sorted_train_data.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,pickup_minute,pickup_time,dropoff_hour,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty
186987,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 20:38:00,2013-01-13 20:58:00,1,1200,9.06,...,38,20.633333,20,2013-01-13,2013-01-13,10.418191,31.254572,2,0.000000,0.000000
136205,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:08:00,2013-01-13 21:10:00,1,120,0.44,...,8,21.133333,21,2013-01-13,2013-01-13,0.376856,11.305694,2,21.362104,0.029670
114412,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:31:00,2013-01-13 21:35:00,1,240,0.65,...,31,21.516667,21,2013-01-13,2013-01-13,0.996570,14.948549,2,497.832494,0.691434
148407,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:39:00,2013-01-13 21:46:00,1,420,2.28,...,39,21.650000,21,2013-01-13,2013-01-13,4.608048,39.497556,2,1872.055354,2.600077
114616,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 22:12:00,2013-01-13 22:15:00,1,180,0.91,...,12,22.200000,22,2013-01-13,2013-01-13,1.231572,24.631431,2,3194.887668,4.437344


In [19]:
sorted_train_data.keys()

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'pickup_date', 'dropoff_date', 'distance', 'speed',
       'day_interval', 'waiting_time', 'waiting_penalty'],
      dtype='object')

## Reinforcement Model

In [20]:
class TaxiWorld():
    def __init__(self):
        self.waiting_penalty = 5.00
        pass
                
    def getReward(self, row):
        return row['total_amount'] - row['waiting_time']/3600*self.waiting_penalty
    
    def set_waiting_penalty(self, penalty):
        self.waiting_penalty = penalty
        
    def getState(self):
        print('Current State')
        return self.s
    
    def setState(self, s):
        self.s = s
        
    def getStateSize(self, trip_and_fare):
        return len(trip_and_fare['zip_code'].unique())
    
    def getZipCodes(self, trip_and_fare):
        return self.zip_codes
    
    def getActionSize(self):
        return len(self.actions)

    def nextAction(self, s):
        print('Calculate Next Action based on state')
        
    def check_Q_table(self, s):
        if s not in self.Q:
            self.Q[s] = dict((action, 0.0) for action in self.actions)
            


In [21]:
env = TaxiWorld()

In [22]:
class TaxiRevenue:
    def __init__(self, env, penalty=5.0):
        self.env = env
        self.env.set_waiting_penalty(penalty)
        self.Q = dict()
    
    def generate_key(self, row):
        return str(row['pickup_latitude']) + '_' + str(row['pickup_longitude']) + '_'  + str(row['pickup_weekday']) + '_' + str(row['day_interval'])
    
    def get_tuples(self, key):
        tuples = key.split('_')
        return {'pickup_latitude': tuples[0],
                   'pickup_longitude': tuples[1],
                   'pickup_weekday': tuples[2],
                   'day_interval': tuples[3]}
    
    def build_Q_table(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            reward = self.env.getReward(row)
            if key not in self.Q:
                self.Q[key] = [reward,1]
            else:
                self.Q[key] = [self.Q[key][0] +reward, self.Q[key][1] + 1]
                
    def greedy(self, s):
        return np.argmax(self.Q[s[0]]) 

    def epsilon_greed(self, epsilon, s):
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_a)
        else:
            return self.greedy(s)
        
    def train(self, trip_and_fare):
        self.build_Q_table(trip_and_fare)
        return self.Q
        
    def test(self):
        return self.env.getState()

In [23]:
taxiRevenue = TaxiRevenue(env)
_Q = taxiRevenue.train(sorted_train_data)

In [24]:
_Q

{'40.77409_-73.874542_6_2': [38.3, 1],
 '40.78017_-73.976936_6_2': [4.470330411176281, 1],
 '40.747021_-73.984673_6_2': [4.808565980259543, 1],
 '40.742596_-73.98234599999998_6_2': [7.899923119205074, 1],
 '40.729671_-73.98984499999997_6_2': [2.0626560167123538, 1],
 '40.738003000000006_-73.996307_6_2': [3.861915298723326, 1],
 '40.772236_-73.979004_6_2': [27.95369525326725, 1],
 '40.688484_-73.992889_6_2': [2.5693286710806205, 1],
 '40.68996_-73.994514_6_2': [16.023005277402678, 1],
 '40.777077_-73.946388_6_2': [9.195729931191698, 1],
 '40.74847000000001_-73.988823_0_3': [9.0, 1],
 '40.758125_-73.971802_0_3': [5.043033481905504, 1],
 '40.645481_-73.77636_3_3': [65.3, 1],
 '40.730011_-73.983566_6_3': [13.7, 1],
 '40.74181_-73.993576_6_3': [35.13752908649161, 1],
 '40.786541_-73.942551_6_0': [-5.521481120098947, 1],
 '40.761356_-73.97953000000003_6_0': [-14.231151483324219, 1],
 '40.774097_-73.874466_6_2': [-68.25425596597483, 1],
 '40.76960800000001_-73.960724_6_2': [-96.39363181387989

## Clean data

In [32]:
print(sorted_train_data.shape)
sorted_train_data.keys()


(200001, 34)


Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'pickup_date', 'dropoff_date', 'distance', 'speed',
       'day_interval', 'waiting_time', 'waiting_penalty'],
      dtype='object')

In [33]:
new_df = copy(sorted_train_data[(sorted_train_data.distance != 0) & (sorted_train_data.distance <= 500)])
new_df.shape

(193453, 34)

## Introduce Zip Code in the Historical Data

In [34]:
# trip_data_with_zip_code = copy(sorted_train_data)
trip_data_with_zip_code = copy(new_df)
trip_data_with_zip_code['pickup_zipcode'] = None
trip_data_with_zip_code['dropoff_zipcode'] = None

In [35]:
trip_data_with_zip_code[trip_data_with_zip_code.pickup_zipcode == None].shape

(0, 36)

In [36]:
# trip_data_with_zip_code['pickup_latlng'] = trip_data_with_zip_code.apply(lambda row: '{},{}'.format(row['pickup_latitude'], row['pickup_longitude']), axis=1)
# trip_data_with_zip_code['dropoff_latlng'] = trip_data_with_zip_code.apply(lambda row: '{},{}'.format(row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

In [37]:
trip_data_with_zip_code.keys()

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'pickup_date', 'dropoff_date', 'distance', 'speed',
       'day_interval', 'waiting_time', 'waiting_penalty', 'pickup_zipcode',
       'dropoff_zipcode'],
      dtype='object')

In [38]:
trip_data_with_zip_code.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,dropoff_hour,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty,pickup_zipcode,dropoff_zipcode
186987,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 20:38:00,2013-01-13 20:58:00,1,1200,9.06,...,20,2013-01-13,2013-01-13,10.418191,31.254572,2,0.000000,0.000000,None,None
136205,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:08:00,2013-01-13 21:10:00,1,120,0.44,...,21,2013-01-13,2013-01-13,0.376856,11.305694,2,21.362104,0.029670,None,None
114412,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:31:00,2013-01-13 21:35:00,1,240,0.65,...,21,2013-01-13,2013-01-13,0.996570,14.948549,2,497.832494,0.691434,None,None
148407,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:39:00,2013-01-13 21:46:00,1,420,2.28,...,21,2013-01-13,2013-01-13,4.608048,39.497556,2,1872.055354,2.600077,None,None
114616,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 22:12:00,2013-01-13 22:15:00,1,180,0.91,...,22,2013-01-13,2013-01-13,1.231572,24.631431,2,3194.887668,4.437344,None,None


In [39]:
# def add_zip_code(trip_data):
#     pickup_zipcode_list = []
#     dropoff_zipcode_list = []
#     for index,row in trip_data.T.iteritems():
#         location = geolocator.reverse(row['pickup_latlng'])
#         if 'address' in location.raw and 'postcode' in location.raw['address']:
#             zipcode = location.raw['address']['postcode']
#         else:
#             zipcode = None
#         pickup_zipcode_list.append(zipcode)
        
#         location = geolocator.reverse(row['dropoff_latlng'])
#         if 'address' in location.raw and 'postcode' in location.raw['address']:
#             zipcode = location.raw['address']['postcode']
#         else:
#             zipcode = None
#         dropoff_zipcode_list.append(zipcode)
#     trip_data['pickup_zipcode'] = pickup_zipcode_list
#     trip_data['dropoff_zipcode'] = dropoff_zipcode_list

In [40]:
# documentation: https://pythonhosted.org/uszipcode/#by-coordinate
def add_zip_code_2(trip_data):
    pickup_zipcode_list = []
    dropoff_zipcode_list = []
    for index,row in trip_data.T.iteritems():
        pickup_res = findzip.by_coordinate(row['pickup_latitude'], row['pickup_longitude'], radius=10, returns=1)
        if(len(pickup_res) > 0):
            zipcode = pickup_res[0]["Zipcode"]
        else:
#             zipcode = None
            zipcode = 'NaN'
        pickup_zipcode_list.append(zipcode)

        dropoff_res = findzip.by_coordinate(row['dropoff_latitude'], row['dropoff_longitude'], radius=10, returns=1)
        if(len(dropoff_res) > 0):
            zipcode = dropoff_res[0]["Zipcode"]
        else:
#             zipcode = None 
            zipcode = 'NaN'
        dropoff_zipcode_list.append(zipcode)

    trip_data['pickup_zipcode'] = pickup_zipcode_list
    trip_data['dropoff_zipcode'] = dropoff_zipcode_list

In [101]:
# adf= trip_data_with_zip_code[:10].copy()
# add_zip_code_2(adf)
# adf

In [41]:
add_zip_code_2(trip_data_with_zip_code)
trip_data_with_zip_code.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,dropoff_hour,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty,pickup_zipcode,dropoff_zipcode
186987,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 20:38:00,2013-01-13 20:58:00,1,1200,9.06,...,20,2013-01-13,2013-01-13,10.418191,31.254572,2,0.000000,0.000000,11371,10025
136205,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:08:00,2013-01-13 21:10:00,1,120,0.44,...,21,2013-01-13,2013-01-13,0.376856,11.305694,2,21.362104,0.029670,10023,10024
114412,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:31:00,2013-01-13 21:35:00,1,240,0.65,...,21,2013-01-13,2013-01-13,0.996570,14.948549,2,497.832494,0.691434,10016,10168
148407,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:39:00,2013-01-13 21:46:00,1,420,2.28,...,21,2013-01-13,2013-01-13,4.608048,39.497556,2,1872.055354,2.600077,10010,10278
114616,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 22:12:00,2013-01-13 22:15:00,1,180,0.91,...,22,2013-01-13,2013-01-13,1.231572,24.631431,2,3194.887668,4.437344,10003,10010


In [42]:
trip_data_with_zip_code.shape

(193453, 36)

In [43]:
trip_data_with_zip_code[trip_data_with_zip_code['pickup_zipcode'] == 'NaN'].shape

(9, 36)

In [44]:
trip_data_with_zip_code[trip_data_with_zip_code['dropoff_zipcode'] == 'NaN'].shape

(10, 36)

In [48]:
# These are garbage value. Look at the 'distance' column.
# Only 2% of total data.
adf = copy(trip_data_with_zip_code[(trip_data_with_zip_code['pickup_zipcode'] != 'NaN') & (trip_data_with_zip_code['dropoff_zipcode'] != 'NaN')])
adf.shape

# new_df = copy(sorted_train_data[(sorted_train_data.distance != 0) & (sorted_train_data.distance <= 500)])
# new_df.shape

(193442, 36)

In [57]:
adf.to_csv('clean_data.csv', index=False )

In [58]:
check_data = pd.read_csv('clean_data.csv')

In [59]:
check_data.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,dropoff_hour,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty,pickup_zipcode,dropoff_zipcode
0,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 20:38:00,2013-01-13 20:58:00,1,1200,9.06,...,20,2013-01-13,2013-01-13,10.418191,31.254572,2,0.000000,0.000000,11371,10025
1,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:08:00,2013-01-13 21:10:00,1,120,0.44,...,21,2013-01-13,2013-01-13,0.376856,11.305694,2,21.362104,0.029670,10023,10024
2,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:31:00,2013-01-13 21:35:00,1,240,0.65,...,21,2013-01-13,2013-01-13,0.996570,14.948549,2,497.832494,0.691434,10016,10168
3,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 21:39:00,2013-01-13 21:46:00,1,420,2.28,...,21,2013-01-13,2013-01-13,4.608048,39.497556,2,1872.055354,2.600077,10010,10278
4,8FA6FBF2D595A1080BCCE03FC2C213F7,0008B3E338CE8C3377E071A4D80D3694,VTS,1,NaN,2013-01-13 22:12:00,2013-01-13 22:15:00,1,180,0.91,...,22,2013-01-13,2013-01-13,1.231572,24.631431,2,3194.887668,4.437344,10003,10010


In [62]:
check_data.keys()

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'pickup_date', 'dropoff_date', 'distance', 'speed',
       'day_interval', 'waiting_time', 'waiting_penalty', 'pickup_zipcode',
       'dropoff_zipcode'],
      dtype='object')

In [66]:
print(check_data['medallion'].nunique())
print(historical_data['medallion'].nunique())

6927
13426
